# SMO

上面求对偶问题的时候我们说到SMO算法,但是没有具体说明,现在我们来看看.

$\underset{a}{max}(\sum_{i=1}^{n}a_i-\frac{1}{2}\sum_{i,j=1}^{n}a_ia_jy_iy_j<x_i,x_j>)$

s.t., $0\leqslant a_i \leqslant C,i=1,...,n$

$\sum_{i=1}^{n}a_iy_i=0$

等价于(这个等价不好推导建议直接知道结果去细想一下):

$\underset{a}{min}\Psi(\vec{a})=\underset{a}{min}\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}Y_iY_jK(\vec{x_i},\vec{x_j})a_ia_j-\sum_{i=1}^{N}a_i$

s.t., $0\leqslant a_i \leqslant C,\forall {i}$

$\sum_{i=1}^{N}a_iy_i=0$

1998年，Microsoft Research的John C. Platt在论文[《Sequential Minimal Optimization：A Fast Algorithm for Training Support Vector Machines》](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-98-14.pdf)中提出针对上述问题的解法：SMO算法，它很快便成为最快的二次规划优化算法，特别是在针对线性SVM和数据稀疏时性能更优。

#### 1.SMO算法推导

首先定义特征到结果的输出函数:

$u=\vec{W}\cdot\vec{X}-b$

这个u与我们之前定义的$f(x)=w^{T}x+b$是一样的

接着重新定义一下原始的优化问题:

$\underset{w,b}{min}\frac{1}{2}||\vec{w}||$ s.t. $y_i(\vec{w}\cdot\vec{x_i}-b) \geqslant1,\forall {i}$

求导得到:

$\vec{w}=\sum_{i=1}^{N}y_ia_i\vec{x_i},b=\vec{w}\cdot\vec{x_k}-y_k$ for som $a_k>0$

带入 $u=\vec{W}\cdot\vec{X}-b$得到,$u=\sum_{j=1}^{N}y_ja_jK(\vec{x_j},\vec{x})-b$

引入拉格朗日乘子转换后:

$\underset{a}{min}\Psi(\vec{a})=\underset{a}{min}\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}Y_iY_jK(\vec{x_i},\vec{x_j})a_ia_j-\sum_{i=1}^{N}a_i$

s.t., $ a_i \geqslant 0,\forall {i}$

$\sum_{i=1}^{N}a_iy_i=0$

加入松弛变量后,模型修改为:

$\underset{w,b,\xi}{min}\frac{1}{2}||\vec{w}||^{2}+C\sum_{i=1}^{N}$ s.t. $y_i(\vec{w}\cdot\vec{x_i}-b)\geqslant 1-\xi_i,\forall {i},0\leqslant a_i \leqslant C,\forall {i}$

最终我们的问题变成为:

$\underset{a}{min}\Psi(\vec{a})=\underset{a}{min}\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}Y_iY_jK(\vec{x_i},\vec{x_j})a_ia_j-\sum_{i=1}^{N}a_i$

s.t., $0\leqslant a_i \leqslant C,\forall {i}$

$\sum_{i=1}^{N}a_iy_i=0$

下面要解决的问题就是在上述$a_i=\{a_1,a_2,...,a_n\}$上求目标函数的最小值.那么我们按照之前的想法我们去固定$a_1$以外的所有参数,然后在$a_1$上去求极值,这样可行嘛?很显然是不行的,因为我们现在多了一个约束条件:

$\sum_{i=1}^{N}a_iy_i=0$

那么一旦固定$a_1$以外的所有参数,那么$a_1$也就固定下来了

$a_1y_1=-\sum_{i=2}^{N}a_iy_i$

所以我们现在需要固定除了$a_1,a_2$以外的所有参数这样的话$a_1,a_2$就不是死的了，也就是说目标函数只是关于$a_1,a_2$的函数,这样不断的在一堆乘子中去抽取两个求解,不断迭代,最终找到原问题的解.

因为我们已经固定了除了$a_1,a_2$,以外的所有乘子，所以除了$a_1,a_2$以外的所有乘子之和是一个常数,并且由于限制条件$\sum_{i=1}^{N}a_iy_i=0$的存在,那么形式如下:

$a_iy_i+a_2y_2 + \sum_{i=3}^{N}a_iy_i=0 \Rightarrow a_iy_i+a_2y_2 + \zeta =0$

$a_iy_i+a_2y_2=\zeta $

$\zeta $是一个常数

那么对于原对偶问题的子问题的目标函数可以表达为(这个形式下面会阐述):

$\Psi=\frac{1}{2}K_{11}a_1^{2}+\frac{1}{2}K_{22}a_{2}^{2}+sK_{12}a_1a_2+y_1a_1v_1+y_2a_2v_2-a_2+\Psi_{constant} $

其中

$K_{ij}=K(\vec{x_i},\vec{x_j}),$

$v_i=\sum_{j=3}^{N}y_ja_jK_{ij}=u_i+b-y_1a_1K_{1i}-y_2a_2K_{2i}$

为了解决这个子问题,首要的问题就是如何选取这个$a_1,a_2$，根据KKT条件可以得出目标函数中$a_i$的取值的意义:

$a_i=0 \Leftrightarrow y_iu_i \Rightarrow 1,$

$0<a_i<C \Leftrightarrow  y_iu_i=1,$

$a_i=C \Leftrightarrow  y_iu_i \leqslant 1.$

这里啊$a_i$还是拉格朗日乘子:

1.对于第一种情况,表明$a_i$是正常分类,在间隔边界内部或者叫间隔边界后面(我们知道正确分类的点$y_if(x_i) \geqslant 0$);

2.对于第二种情况,表明$a_i$是支持向量,在间隔边界上;

3.对于第三种情况,表明$a_i$是在两条间隔边界之间;

而最优解需要满足KKT条件,即上述3个条件都得满足,那么以下几种情况将会出现不满足:

1.$ y_iu_i \leqslant 1.$ 但是$a_i<C$是不满足的

2.$y_iu_i \Rightarrow 1$,但是$a_i>0$是不满足的

3.$y_iu_i =1$,但是$a_i=0$或$a_i=C$是不满足的

也就是说如果存在不满足KKT条件的$a_i$,那么需要更新这些$a_i$,这是第一个约束条件.

我们现在选择两个乘子$a_1,a_2$,他们在更新前分别是$a_1^{old},a_2^{old}$，更新后是$a_1^{new},a_2^{new}$,由于第二个约束条件的存在(也就是我们上面说的$\sum_{i=1}^{N}a_iy_i=0$),所以我们要保证:

$a_1^{new}y_1+a_2^{new}y_2=a_1^{old}y_1+a_2^{old}y_2=\zeta$

这里的$\zeta$是一个常数,且上述的等式的意思是前面两部分都是等于一个常数

因为两个因子不好同时求解,所以我们先求第二个乘子的解(当然要先求第一个也是可以的$a_2$的解$a_2^{new}$,得到$a_2$的解$a_2^{new}$后,再用$a_2^{new}$去表示$a_1$的解$a_1^{new}$.

为了求解$a_2^{new}$,需要先确定$a_2^{new}$的取值范围。假设它的上下边界分别为H和L,那么有:

$L\leqslant a_2^{new} \leqslant H$

接下来综合$0 \leqslant a_i \leqslant C,i=1,...,n$和$a_1^{new}y_1+a_2^{new}y_2=a_1^{」old}y_1+a_2^{old}y_2=\zeta$这两个约束条件求$a_2^{new}$的取值范围

因为我们的y取值只有1或者-1

那么在当y1!=y2的时候，根据$a_1^{new}y_1+a_2^{new}y_2=a_1^{old}y_1+a_2^{old}y_2=\zeta$可得:

- $a_1^{old}-a_2^{old}=\zeta$或者$a_2^{old}-a_1^{old}=\zeta$

我们使用$0 \leqslant a_i \leqslant C,i=1,...,n$进行取值画图:
![](picture/28.png)

如果$a_2<a_1$,很明显$a_2$的上限是$(C,C-\zeta)$中的$C-\zeta$.

如果$a_2>a_1$,很明显$a_2$的上限是$(C+\zeta,C)$中的$C$.

那么两个综合一下,用一个式子进行表示就是$H=min(C,C-\zeta)$

也就是说如果$a_2<a_1$时,我们取的是较小的$C-\zeta$,如果$a_2>a_1$我们也是去较小的$C$

同样我们来看看$a_2$的下限:

如果$a_2<a_1$,很明显$a_2$的下限是0.

如果$a_2>a_1$,很明显$a_2$的下限是$-\zeta$.

那么两个综合一下,用一个式子进行表示就是$L=min(0,-\zeta)$

那么在y1=y2的时候

同样根据$a_1^{new}y_1+a_2^{new}y_2=a_1^{old}y_1+a_2^{old}y_2=\zeta$可得:

$a_1^{old}+a_2^{old}=\zeta$,所以$L=max(0,\zeta-C),H=min(C,\zeta)$

![](picture/29.png)

如此,根据y1和y2异号或者同号,可以得出$a_2^{new}$的上下界分别是:

$\left\{\begin{matrix}
L=max(0,a_2^{old}-a_1^{old}),H=min(C,C+a_2^{old}-a_1^{old}) &if y1\neq y_2 \\ 
L=max(0,a_2^{old}+a_1^{old}-C),H=min(C,a_2^{old}+a_1^{old}) &if y_1  = y_2 
\end{matrix}\right.$

回顾下第二个约束条件$a_1^{new}y_1+a_2^{new}y_2=a_1^{old}y_1+a_2^{old}y_2=\zeta$,令两边乘以y1,可得

$a_1+sa_2=a_1^{*}+sa_2^{*}=w$

其中$w=-y_1\sum_{i=3}^{n}y_ia_i^{*}$

因此$a_1$可以用$a_2$表示,$a_1=w-s*a_2$,从而把子问题的目标函数转换为只含$a_2$的问题:

$\Psi=\frac{1}{2}K_{11}a_1^{2}+\frac{1}{2}K_{22}a_{2}^{2}+sK_{12}a_1a_2+y_1a_1v_1+y_2a_2v_2-a_2+\Psi_{constant} $

对$a_2$求导:

$\frac{\partial \Psi}{\partial a_2}=-sK_{11}(w-sa_2)+K_{22}a_2-K_{12}a_2+sK_{12}(w-sa_2)-y_2v_1+s+y_2v_2-1=0$

简化下

$a_2(K_{11}+K_{22}-2K_{12})=s(K_{11}-K_{12})w+y_2(v_1-v_2)+1-s$


然后将$s=y_1*y_2,a_1+sa_2=a_1^{*}+sa_2^{*}=w$,

$K_{ij}=K(\vec{x_i},\vec{x_j})$,

$v_i=\sum_{j=3}^{N}y_ja_jK_{ij}=u_i+b-y_1a_1K_{1i}-y_2a_2K_{2i}$

带入上式可得(由于这里我们还没有考虑到$ 0\leqslant a_2 \leqslant C $,所以这个解是未经过裁剪的我们用unc表示):

$a_2^{new,unc}(K_{11}+K_{22}-2K_{12})=a_2^{old}(K_{11}+K_{22}-2K_{12})+y_2(u_1-u_2+y_2-y_1)$

令$E_i=u_i-y_i$(表示真实值与预测值之差),$\eta =K(\vec{x_1},\vec{x_1})+K(\vec{x_2},\vec{x_2})-2K(\vec{x_1},\vec{x_2})$

然后两边同时除上$\eta$,得到一个关于单变量$a_2$的解:

$a_2^{new,unc}=a_2^{old}+\frac{y_2(E_1-E_2)}{\eta}$

那么没有考虑约束条件的单变量$a_2$就解出来了,现在加入条件$ 0\leqslant a_2 \leqslant C $的解为:

$a_2^{new}=\left\{ \begin{matrix}
 H,&a_2^{new,unc}>H 
\\   a_2^{new,unc},&L\leqslant a_2^{new,unc}\leqslant H
\\  L,&a_2^{new,unc} \leqslant L 
\end{matrix}\right.$

求解出$a_2^{new}$后那么$a_1^{new}=a_1^{old}+y_1y_2(a_2^{old}-a_2^{new})$



那么如何选择$a_1,a_2$呢？

- 对于$a_1$,即第一个乘子,我们可以通过刚刚说的那3中不满足KKT条件来找;
- 对于第二个乘子$a_2$，可以寻找条件满足$max|E_i-E_j|$的乘子

在b满足下述条件:

$\left\{ \begin{matrix}
 b_1&if \ 0<a_1^{new}<C,
\\  b_2&if \ 0<a_2^{new}<C
\\  \frac{(b_1+b_2)}{2}&otherwise
\end{matrix}\right.$

![](picture/30.png)

 且每次更新完两个乘子的优化后，都需要再重新计算b，及对应的Ei值。

最后更新出来所有的$a_i$,y,b这样模型就出来了

$f(x)=\sum_{i=1}^{n}a_iy_i<x_i,x>+b$

## 总结SMO

SMO的主要步骤如下(图来自博主v_JULY_v):

![](picture/31.png)

![](picture/32.png)